In [47]:
import pandas as pd
import lightgbm as lgb
from datetime import datetime
#from lightgbm import LGBMRegressor

In [ ]:
try:
    FILES
    FILE = FILES[4]
    FOLDER
except NameError:
    FILE = ['05_sell_in_pred.parquet',
            '05_sell_in_pred_futuro.parquet']
    FOLDER = 'DefaultFolder'


print(f"------------------------------COMIENZA-----------------------------\n[{FOLDER}/{FILE}")

------------------------------COMIENZA-----------------------------
[DefaultFolder/['01_sell_in_group.parquet', '01_prod_stats.parquet', '01_prod_data.parquet', '01_stock_data.parquet']


In [49]:
# df_train = pd.read_parquet('sell_in_train.parquet')
# df_test = pd.read_parquet('sell_in_test.parquet')
# df_futuro = pd.read_parquet('sell_in_futuro.parquet')
df_train = pd.read_parquet(f"{FOLDER}/{FILES[3][0]}")
df_test = pd.read_parquet(f"{FOLDER}/{FILES[3][1]}")
df_futuro = pd.read_parquet(f"{FOLDER}/{FILES[3][2]}")

print(f"df_train: {df_train.shape}")
print(f"df_test: {df_test.shape}")
print(f"df_futuro: {df_futuro.shape}")

df_train: (20789, 179)
df_test: (780, 179)
df_futuro: (780, 179)


In [50]:
categorical_features = df_train.select_dtypes(['category']).columns.tolist()
for col in categorical_features:
    print(f"convertida {col}")
    df_train[col] = df_train[col].cat.codes
    df_test[col] = df_test[col].cat.codes
    df_futuro[col] = df_futuro[col].cat.codes

convertida yearquarter
convertida cat1
convertida cat2
convertida cat3
convertida brand


In [51]:
X_train = df_train.iloc[:,:-1]
X_test = df_test.iloc[:,:-1]
X_futuro = df_futuro.iloc[:,:-1]

y_train = df_train.iloc[:,-1]
y_test = df_test.iloc[:,-1]
y_futuro = df_futuro.iloc[:,-1]

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"X_futuro: {X_futuro.shape}")

print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")
print(f"y_futuro: {y_futuro.shape}")

X_train: (20789, 178)
X_test: (780, 178)
X_futuro: (780, 178)
y_train: (20789,)
y_test: (780,)
y_futuro: (780,)


In [52]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)
futuro_data = lgb.Dataset(X_futuro, label=y_futuro)

params = {
    'objective': 'regression',
    'metric': {'l2', 'rmse'},
    'boosting_type': 'gbdt',
    'num_leaves': 40,
#    'max_depth': -1,
    'learning_rate': 0.02,
#    'feature_fraction': 0.4
}

model = lgb.train(params, train_data, num_boost_round=1000, valid_sets=[train_data, test_data])

#y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred_futuro = model.predict(X_futuro, num_iteration=model.best_iteration)

#example_row = X_test.iloc[0]
#example_pred = model.predict(example_row.values.reshape(1, -1), num_iteration=model.best_iteration)
#print(f'Prediction for example row: {example_pred[0]}')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017999 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25125
[LightGBM] [Info] Number of data points in the train set: 20789, number of used features: 172
[LightGBM] [Info] Start training from score -0.057580


In [53]:
final = X_test
final['tn_futuro'] = y_test
final['tn_prediccion'] = y_pred

In [54]:
#prod_stats = pd.read_parquet('prod_stats.parquet')
prod_stats = pd.read_parquet(f"{FOLDER}/{FILES[0][1]}")
prod_stats = prod_stats[['product_id', 'average_tn','std_dev_tn']]
final = final.merge(prod_stats, how='left', on='product_id')
final.columns

Index(['product_id', 'periodo', 'primer_periodo', 'ultimo_periodo', 'tn_norm',
       'periodo_dt', 'mes', 'quarter', 'month_in_quarter', 'year',
       ...
       'prop_product_yearquarter_cat1', 'prop_product_yearquarter_cat2',
       'prop_product_yearquarter_brand', 'prop_product_year_cat1',
       'prop_product_year_cat2', 'prop_product_year_brand', 'tn_futuro',
       'tn_prediccion', 'average_tn', 'std_dev_tn'],
      dtype='object', length=182)

In [55]:
final['tn_futuro_real'] = (final['tn_norm'] + final['tn_futuro']) * final['std_dev_tn'] + final['average_tn']
final['tn_prediccion_real'] = (final['tn_norm'] + final['tn_prediccion']) * final['std_dev_tn'] + final['average_tn']

In [56]:
final.head()

,product_id,periodo,primer_periodo,ultimo_periodo,tn_norm,periodo_dt,mes,quarter,month_in_quarter,year,...,prop_product_yearquarter_brand,prop_product_year_cat1,prop_product_year_cat2,prop_product_year_brand,tn_futuro,tn_prediccion,average_tn,std_dev_tn,tn_futuro_real,tn_prediccion_real
0,20001,201912,201701,201912,0.356686,201912,12,4,3,2019,...,24.683874,0.702526,1.252843,6.476876,NaN,-0.874609,1398.344322,298.145460,NaN,1243.927851
1,20002,201912,201701,201912,0.256522,201912,12,4,3,2019,...,4.739874,0.507655,0.905322,1.198157,NaN,-0.756614,1009.368178,303.834835,NaN,857.422856
2,20003,201912,201701,201912,0.011975,201912,12,4,3,2019,...,7.660107,1.258237,1.673822,2.237771,NaN,-0.853452,889.004243,292.036581,NaN,643.262221
3,20004,201912,201701,201912,-0.150213,201912,12,4,3,2019,...,5.474930,0.899303,1.196336,1.599409,NaN,-0.750473,671.615383,224.450085,NaN,469.456387
4,20005,201912,201701,201912,-0.233451,201912,12,4,3,2019,...,5.091663,0.836348,1.112587,1.487444,NaN,-0.321491,644.200514,218.273222,NaN,523.071525


In [57]:
#final.to_parquet('sell_in_pred.parquet', index=False)
final.to_parquet(f'{FOLDER}/{FILE[0]}', index=False)

In [58]:
final_futuro = X_futuro
final_futuro['tn_futuro'] = y_futuro
final_futuro['tn_prediccion'] = y_pred_futuro

final_futuro = final_futuro.merge(prod_stats, how='left', on='product_id')
final_futuro.columns

final_futuro['tn_futuro_real'] = (final_futuro['tn_norm'] + final_futuro['tn_futuro']) * final_futuro['std_dev_tn'] + final_futuro['average_tn']
final_futuro['tn_prediccion_real'] = (final_futuro['tn_norm'] + final_futuro['tn_prediccion']) * final_futuro['std_dev_tn'] + final_futuro['average_tn']

#final_futuro.to_parquet('sell_in_pred_futuro.parquet', index=False)
final_futuro.to_parquet(f'{FOLDER}/{FILE[1]}', index=False)

In [59]:
#estado_control = f"05_lightgbm Terminado - {nombrefile} - {datetime.now()}"

In [ ]:
print(f" {FOLDER}/{FILE}\n------------------------------FINALIZA-----------------------------")

 DefaultFolder/['01_sell_in_group.parquet', '01_prod_stats.parquet', '01_prod_data.parquet', '01_stock_data.parquet']
------------------------------FINALIZA-----------------------------
